1. Load Bayesian CNN model as Annotator
2. Active Learning will query the samples based on its acquistion function
3. Queried samples will be sent to BCNN for labelling.
4. Only those labels will be submitted whose labels are confidently returned by the BCNN model. 
5. Active Learning Model will be retrained.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as tfk
tfkl=tfk.layers
from tensorflow.keras.models import Model

In [ ]:
num_base_filters=32
l2_reg=1e-4  #1e-4
dropout_rate=0.15  # for MC-Dropout
learning_rate=1e-4
bs=64

In [ ]:
inputs=tfkl.Input((224,224,3))
#Block1
x=tfkl.Conv2D(filters=num_base_filters,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(inputs)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.MaxPooling2D(pool_size=3, strides=(2, 2), padding="same")(x)
#Block2
x=tfkl.Conv2D(filters=num_base_filters*2,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters*2,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.MaxPooling2D(pool_size=3, strides=(2, 2), padding="same")(x)
#Block3
x=tfkl.Conv2D(filters=num_base_filters*4,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters*4,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.MaxPooling2D(pool_size=3, strides=(2, 2), padding="same")(x)
# Block 4
x=tfkl.Conv2D(filters=num_base_filters*8,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters*8,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters*8,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.MaxPooling2D(pool_size=5, strides=(3, 3), padding="same")(x)
# Block 5
x=tfkl.Conv2D(filters=num_base_filters*8,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters*8,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Conv2D(filters=num_base_filters*8,kernel_size=3,strides=(1, 1),padding="same",kernel_regularizer=tfk.regularizers.l2(l2_reg))(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(dropout_rate)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.MaxPooling2D(pool_size=5, strides=(3, 3), padding="same")(x)
#x=tfkl.Lambda(lambda y: tfk.backend.concatenate([tfkl.GlobalAvgPool2D()(y),tfkl.GlobalMaxPool2D()(y)], axis=1))(x)
x=tfkl.Flatten()(x)
# Fully-connected
x=tfkl.Dense(2048)(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(0.5)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Dense(512)(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(0.5)(x,training=True)
x=tfkl.BatchNormalization()(x)
x=tfkl.Dense(128)(x)
x=tfkl.LeakyReLU()(x)
x=tfkl.Dropout(0.2)(x,training=True)
x=tfkl.BatchNormalization()(x)
outputs=tfkl.Dense(5,activation='softmax')(x)
model=Model(inputs, outputs)

In [ ]:
model.load_weights("/content/drive/My Drive/best_model_multi_class_ce.h5")

In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 224, 224, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 32)     

In [ ]:
intermediate_model=Model(inputs=model.inputs,outputs=model.get_layer("leaky_re_lu_11").output)

In [ ]:
for layer in model.layers:
  layer.trainable=False
for layer in intermediate_model.layers:
  layer.trainable=False

In [ ]:
print(intermediate_model.summary())

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 32)      896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 224, 224, 32)      0         
_________________________________________________________________
dropout (Dropout)            (None, 224, 224, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 32)      128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 32)      9248      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 224, 224, 32)     

In [ ]:
X_O=[]
y_O=[]
X_F=[]
y_F=[]
import glob
import cv2
def make_arrays(dir):
  for folders in dir:
    for folder in folders:
      label=int(folder[len(folder)-1])
      #print(folder)
      fldr= folder.split("/")
      fldr=fldr[len(fldr)-2]
      if (fldr=="test" and label==0) or (fldr=="test" and label==1) or (fldr=="test" and label==2):
        print(fldr)
        print(label)
        continue;
      else:
        files=glob.glob(folder + "/*")
        #print(len(files))
        for file in files:
          res=cv2.cvtColor(cv2.resize(cv2.imread(file),(224,224)), cv2.COLOR_BGR2RGB)/255.0
          X_O.append(res)
          res=np.expand_dims(res,axis=0)
          X_F.append(np.squeeze(intermediate_model(res,training=False),axis=0))
          y_O.append(label)
          y_F.append(label)
  return np.array(X_O),np.array(y_O),np.array(X_F),np.array(y_F)

In [ ]:
filesv=glob.glob("/content/drive/My Drive/Multiclass Dataset New/val/*")
filest=glob.glob("/content/drive/My Drive/Multiclass Dataset New/test/*")
import tensorflow as tf
X_O,y_O,X_F,y_F=make_arrays([filesv,filest])
y_O=tf.keras.utils.to_categorical(y_O, num_classes=5, dtype='float32')
y_F=tf.keras.utils.to_categorical(y_F, num_classes=5, dtype='float32')
print("X Orig : ",X_O.shape)
print("y Orig : ",y_O.shape)
print("X Feat : ",X_F.shape)
print("y_Feat : ",y_F.shape)
# np.save("/content/X_F.npy",X_F)
# np.save("/content/y_F.npy",y_F)
# np.save("/content/X_O.npy",X_O)
# np.save("/content/y_O.npy",y_O)

test
0
test
1
test
2
X Orig :  (2238, 224, 224, 3)
y Orig :  (2238, 5)
X Feat :  (2238, 10, 10, 256)
y_Feat :  (2238, 5)


In [ ]:
p=np.random.permutation(range(0,len(X_F)))
X_O=X_O[p]
y_O=y_O[p]
X_F=X_F[p]
y_F=y_F[p]

In [ ]:
n_inctances=300
train_idx = np.random.choice(range(X_F.shape[0]), size=n_inctances, replace=False)
X_initial=X_F[train_idx]
y_initial=y_F[train_idx]
print(X_initial.shape)    
print(y_initial.shape)

(300, 10, 10, 256)
(300, 5)


In [ ]:
n_initial=100
p=np.random.choice(range(len(X_initial)),size=n_initial, replace=False)
Xl1=X_initial[p]
yl1=y_initial[p]
X_initial = np.delete(X_initial, p, axis=0)
y_initial = np.delete(y_initial, p, axis=0)
print(Xl1.shape)
print(yl1.shape)
p=np.random.choice(range(len(X_initial)),size=n_initial, replace=False)
Xl2=X_initial[p]
yl2=y_initial[p]
X_initial = np.delete(X_initial, p, axis=0)
y_initial = np.delete(y_initial, p, axis=0)
print(Xl2.shape)
print(yl2.shape)
p=np.random.choice(range(len(X_initial)),size=n_initial, replace=False)
Xl3=X_initial[p]
yl3=y_initial[p]
X_initial = np.delete(X_initial, p, axis=0)
y_initial = np.delete(y_initial, p, axis=0)
print(Xl3.shape)
print(yl3.shape)

(100, 10, 10, 256)
(100, 5)
(100, 10, 10, 256)
(100, 5)
(100, 10, 10, 256)
(100, 5)


In [ ]:
X_O = np.delete(X_O, train_idx, axis=0)
y_O = np.delete(y_O, train_idx, axis=0)
X_F = np.delete(X_F, train_idx, axis=0)
y_F = np.delete(y_F, train_idx, axis=0)
print(X_O.shape)
print(y_O.shape)
print(X_F.shape)
print(y_F.shape)

(1938, 224, 224, 3)
(1938, 5)
(1938, 10, 10, 256)
(1938, 5)


In [ ]:
X_pool= X_F[0:int(0.70*X_F.shape[0])]
X_orig= X_O[0:int(0.70*X_O.shape[0])]
y_O = y_O[0:int(0.70*X_O.shape[0])]
print(X_pool.shape)
print(X_orig.shape)
print(y_O.shape)

(1356, 10, 10, 256)
(1356, 224, 224, 3)
(1356, 5)


In [ ]:
X_test= X_F[int(0.70*X_F.shape[0]):]
y_test = y_F[int(0.70*y_F.shape[0]):]
print(X_test.shape)    
print(y_test.shape)

(582, 10, 10, 256)
(582, 5)


In [ ]:
def focal_loss(gamma=2., alpha=4.):
    gamma = float(gamma)
    alpha = float(alpha)
    def focal_loss_fixed(y_true, y_pred):
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [ ]:
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.models import Model
opt=Adam(learning_rate=0.0001,beta_1=0.9,beta_2=0.999)
def create_keras_model():
  inputs=tfkl.Input((10,10,256))
  x=tfkl.Conv2D(256,3,strides=1,padding='valid',kernel_regularizer=tfk.regularizers.l2(1e-4))(inputs)
  x=tfkl.LeakyReLU()(x)
  x=tfkl.Dropout(0.20)(x,training=True)
  x=tfkl.BatchNormalization()(x)
  x=tfkl.MaxPool2D(pool_size=(2,2),strides=2)(x)
  x=tfkl.Conv2D(256,3,strides=1,padding='same')(x)
  x=tfkl.LeakyReLU()(x)
  x=tfkl.Dropout(0.20)(x,training=True)
  x=tfkl.BatchNormalization()(x)
  x=tfkl.Conv2D(256,3,strides=1,padding='same')(x)
  x=tfkl.LeakyReLU()(x)
  x=tfkl.Dropout(0.20)(x,training=True)
  x=tfkl.BatchNormalization()(x)
  x=tfkl.Flatten()(x)
  x=tfkl.Dense(2048,kernel_regularizer=tfk.regularizers.l2(1e-4))(x)
  x=tfkl.LeakyReLU()(x)
  x=tfkl.Dropout(0.25)(x,training=True)
  x=tfkl.BatchNormalization()(x)
  x=tfkl.Dense(512,kernel_regularizer=tfk.regularizers.l2(1e-4))(x)
  x=tfkl.LeakyReLU()(x)
  x=tfkl.Dropout(0.25)(x,training=True)
  x=tfkl.BatchNormalization()(x)
  x=tfkl.Dense(128,kernel_regularizer=tfk.regularizers.l2(1e-4))(x)
  x=tfkl.LeakyReLU()(x)
  x=tfkl.Dropout(0.25)(x,training=True)
  x=tfkl.BatchNormalization()(x)
  x=tfkl.Dense(5,kernel_regularizer=tfk.regularizers.l2(1e-4),activation='softmax')(x)
  al_model=Model(inputs,x)
  return al_model

In [ ]:
def Uncertainty_Quantification(x,model,num_samples):
  mc_samples=np.array([model(x,training=True) for _ in range(num_samples)])
  mc_samples=np.squeeze(mc_samples,axis=1)
  mean_mc_samples=np.mean(mc_samples,axis=0)
  y_pred=np.argmax(mean_mc_samples)
  ent=-1*np.sum(np.multiply(mean_mc_samples,np.log(mean_mc_samples)),axis=0)
  return ent,mean_mc_samples

In [ ]:
classifier1 = create_keras_model()
classifier1.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
classifier2 = create_keras_model()
classifier2.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])
classifier3 = create_keras_model()
classifier3.compile(loss='categorical_crossentropy',optimizer=opt,metrics=['accuracy'])

In [ ]:
classifier1.fit(Xl1,yl1,epochs=5,verbose=1,batch_size=4,shuffle=True)
classifier2.fit(Xl2,yl2,epochs=5,verbose=1,batch_size=4,shuffle=True)
classifier3.fit(Xl3,yl3,epochs=5,verbose=1,batch_size=4,shuffle=True)

In [ ]:
def query(x,models,query_strategy="vote_entropy_sampling"):
  res=[]
  for mod in models:
    mc_samples=np.array([mod(x) for _ in range(25)])
    mean_mc_samples=np.mean(mc_samples,axis=0)
    res.append(mean_mc_samples)
  res=np.array(res)
  pred=np.argmax(res,axis=2)
  count = (np.arange(res.shape[2])==pred[...,np.newaxis]).sum(axis=0) # np.arang(No_of_classes)....
  vote_p = count/len(models) 
  vote_p=vote_p + 1e-8
  ent= -(vote_p*np.log(vote_p)).sum(axis=1)
  query_index=np.argmax(ent)
  return query_index

In [ ]:
models=[classifier1,classifier2,classifier3]

In [ ]:
print(classifier1.evaluate(X_test,y_test,verbose=0)[1]*100)
print(classifier2.evaluate(X_test,y_test,verbose=0)[1]*100)
print(classifier3.evaluate(X_test,y_test,verbose=0)[1]*100)

57.21649527549744
57.731956243515015
55.498284101486206


In [ ]:
def evaluate(models,x,y):
  y_pred=[]
  count=0;
  for mod in models:
    p_samples=np.array(mod(x,training=True))
    y_pred.append(p_samples)
  y_pred=np.array(y_pred)
  y_pred=np.mean(y_pred,axis=0)
  y_pred=np.argmax(y_pred,axis=1)
  y_true=np.argmax(y,axis=1)
  for i in range(y_pred.shape[0]):
    if y_pred[i]==y_true[i]:
      count+=1
  return count/y_pred.shape[0]*100

In [ ]:
N_QUERIES = 1200
scor=[]
xtr=[]
ytr=[]
index=0
while (index < N_QUERIES):
  index+=1
  p=np.random.permutation(range(X_pool.shape[0]))
  X_pool=X_pool[p]
  X_orig=X_orig[p]
  y_O=y_O[p]
  pool_f=X_pool[0:10]
  pool_o=X_orig[0:10]
  pool_y= y_O[0:10]
  models=[classifier1,classifier2,classifier3]
  query_index=query(pool_f,models,query_strategy="uncertainty_sampling")
  x=np.expand_dims(pool_o[query_index],axis=0)
  # print(x.shape)
  ent,mean_mc_prob=Uncertainty_Quantification(x,model,10)
  y_pred=tf.keras.utils.to_categorical(np.argmax(mean_mc_prob), num_classes=5, dtype='float32')
  if ent < 1.276:
    xtr.append(pool_f[query_index])
    ytr.append(y_O[query_index])
  if index % 16 == 0:
    xtr=np.array(xtr)
    ytr=np.array(ytr)
    bs=16;
    Xl1=np.concatenate((xtr,Xl1),axis=0)
    yl1=np.concatenate((ytr,yl1),axis=0)
    Xl2=np.concatenate((xtr,Xl2),axis=0)
    yl2=np.concatenate((ytr,yl2),axis=0)
    Xl3=np.concatenate((xtr,Xl3),axis=0)
    yl3=np.concatenate((ytr,yl3),axis=0)
    print("New Training Size after Query No. :",index,Xl1.shape[0])
    classifier1.fit(Xl1,yl1,epochs=20,verbose=0,batch_size=bs,shuffle=True)
    classifier2.fit(Xl2,yl2,epochs=20,verbose=0,batch_size=bs,shuffle=True)
    classifier3.fit(Xl3,yl3,epochs=20,verbose=0,batch_size=bs,shuffle=True)
    scr=evaluate(models,X_test,y_test)
    scor.append(scr)
    print("Test Accuracy after Iteration:",index,scr)
    xtr=[]
    ytr=[]
  X_pool = np.delete(X_pool, query_index, axis=0)
  X_orig= np.delete(X_orig, query_index, axis=0)
  y_O= np.delete(y_O, query_index, axis=0)

New Training Size after Query No. : 16 116
Test Accuracy after Iteration: 16 69.24398625429554
New Training Size after Query No. : 32 132
Test Accuracy after Iteration: 32 70.44673539518901
New Training Size after Query No. : 48 148
Test Accuracy after Iteration: 48 73.02405498281786
New Training Size after Query No. : 64 164
Test Accuracy after Iteration: 64 74.91408934707904
New Training Size after Query No. : 80 180
Test Accuracy after Iteration: 80 75.94501718213058
New Training Size after Query No. : 96 196
Test Accuracy after Iteration: 96 75.08591065292096
New Training Size after Query No. : 112 212
Test Accuracy after Iteration: 112 74.74226804123711
New Training Size after Query No. : 128 228
Test Accuracy after Iteration: 128 76.97594501718214
New Training Size after Query No. : 144 244
Test Accuracy after Iteration: 144 75.4295532646048
New Training Size after Query No. : 160 260
Test Accuracy after Iteration: 160 76.80412371134021
New Training Size after Query No. : 176 276

In [ ]:
import numpy as np
np.save("/content/Final_Accu_AL_MCDP_Multi_QBC_ce.npy",scor)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(scor)